In [ ]:
!pip install qiskit scipy qiskit_aer matplotlib qiskit_nature pyscf

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_aer.primitives import Estimator

from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

from numpy.random import standard_normal

from scipy.optimize import minimize

In [ ]:
# https://cccbdb.nist.gov/exp2x.asp?casno=7664393&charge=0
HF_equilibrium_geometry = "H 0.0 0.0 0.0; F 0.0 0.0 0.9168"

layers = 2

In [ ]:
driver = PySCFDriver(atom=HF_equilibrium_geometry)
electronic_structure_problem = driver.run()

second_quantized_Hamiltonian = electronic_structure_problem.hamiltonian.second_q_op()

mapper = JordanWignerMapper()
HF_Hamiltonian = mapper.map(second_quantized_Hamiltonian)

In [ ]:
def number_preserving_ansatz(n_qubits, n_layers, param_name="θ"):
    n_exchange_per_layer = (n_qubits // 2) + (
        (n_qubits - 1) // 2
    )  # even-pairs + odd-pairs

    total_params = n_layers * (n_exchange_per_layer + n_qubits)
    params = ParameterVector(param_name, total_params)

    qc = QuantumCircuit(n_qubits)

    param_index = 0
    for _ in range(n_layers):
        for q in range(n_qubits):
            qc.rz(params[param_index], q)
            param_index += 1

        # Even pairs
        for i in range(0, n_qubits - 1, 2):
            qc.rxx(params[param_index], i, i + 1)
            qc.ryy(params[param_index], i, i + 1)
            param_index += 1

        # Odd pairs
        for i in range(1, n_qubits - 1, 2):
            qc.rxx(params[param_index], i, i + 1)
            qc.ryy(params[param_index], i, i + 1)
            param_index += 1

    return qc, params

In [ ]:
hartree_fock_state = HartreeFock(
    num_spatial_orbitals=6, num_particles=(5, 5), qubit_mapper=mapper
)

In [ ]:
ansatz = QuantumCircuit(12)

ansatz_A, params_A = number_preserving_ansatz(6, 2, param_name="θ_A")
ansatz_B, params_B = number_preserving_ansatz(6, 2, param_name="θ_B")
ansatz.compose(ansatz_A, qubits=[0, 1, 2, 3, 4, 5], inplace=True)
ansatz.compose(ansatz_B, qubits=[6, 7, 8, 9, 10, 11], inplace=True)

In [ ]:
qc = QuantumCircuit(12)

qc.compose(hartree_fock_state, inplace=True)
qc.compose(ansatz, inplace=True)

qc.measure_all()

qc.draw(fold=-1)

In [ ]:
estimator = Estimator()


def estimate_energy(params):
    val = estimator.run(qc, HF_Hamiltonian, parameter_values=params).result().values[0]
    return val

In [ ]:
initial_params = 0.05 * standard_normal(len(params_A) + len(params_B))
optimum = minimize(estimate_energy, x0=initial_params, method="COBYLA")

optimal_params = optimum.x
optimal_qc = qc.assign_parameters(optimal_params)

In [ ]:
optimal_energy = estimate_energy(optimal_params)

print(optimal_energy)